In [4]:

import torch
from screenai.main import ScreenAI

# Create a tensor for the image
image = torch.rand(1, 3, 224, 224)

# Create a tensor for the text
# text = torch.randn(1, 1, 512)
text = torch.randint(0, 20000, (1, 1028), dtype=torch.long)  # Ensure dtype is long


# Create an instance of the ScreenAI model with specified parameters
model = ScreenAI(
    num_tokens = 20000,
    max_seq_len = 1028,
    patch_size=16,
    image_size=224,
    dim=512,
    depth=6,
    heads=8,
    vit_depth=4,
    multi_modal_encoder_depth=4,
    llm_decoder_depth=4,
    mm_encoder_ff_mult=4,
)

# Perform forward pass of the model with the given text and image tensors
out = model(text, image)

# Print the shape of the output tensor
print(f'Image:{image}')
print(f'Text:{text}')
print(f"Output:{out}")

Image shape: torch.Size([1, 196, 512])
torch.Size([1, 1224, 512])
Image:tensor([[[[0.4924, 0.0281, 0.6164,  ..., 0.8618, 0.6266, 0.1216],
          [0.1232, 0.2853, 0.3252,  ..., 0.6083, 0.8591, 0.2687],
          [0.2171, 0.5652, 0.9441,  ..., 0.5477, 0.2376, 0.2095],
          ...,
          [0.4456, 0.8558, 0.8772,  ..., 0.8245, 0.3661, 0.9615],
          [0.9290, 0.2837, 0.8245,  ..., 0.5576, 0.7859, 0.0949],
          [0.2372, 0.5417, 0.5213,  ..., 0.0902, 0.8180, 0.0462]],

         [[0.9899, 0.6287, 0.4743,  ..., 0.1474, 0.3460, 0.1527],
          [0.2519, 0.6791, 0.0676,  ..., 0.2312, 0.5934, 0.1986],
          [0.8409, 0.8834, 0.1183,  ..., 0.3674, 0.7812, 0.5270],
          ...,
          [0.4106, 0.6686, 0.8391,  ..., 0.1932, 0.2391, 0.3995],
          [0.5824, 0.7672, 0.4333,  ..., 0.5844, 0.1902, 0.4470],
          [0.8668, 0.5266, 0.6382,  ..., 0.7829, 0.2651, 0.6186]],

         [[0.6815, 0.6437, 0.8876,  ..., 0.9481, 0.0388, 0.3285],
          [0.6342, 0.9647, 0.8647,  

In [10]:
print(f"Output Shape: {out.shape}")


Output Shape: torch.Size([1, 1224, 512])
